# Week 2 Exercise: Deep Search Assistant
## BY Mougang Thomas Gasmyr from the Wakanda Team

An AI-powered deep search assistant that can search the web in real-time using **Tavily**,
synthesize results, and deliver comprehensive answers.

**Features:**
- **Gradio Blocks UI** with streaming chat
- **Model switching** between GPT, Claude, and Gemini
- **Tavily deep search** tool — the LLM autonomously searches the web to ground its answers
- **Audio input** via microphone (Whisper transcription)
- **Audio validation** — recordings are checked for silence, duration, volume, and sample rate before calling the LLM, saving unnecessary API costs
- **Audio output** via TTS (text-to-speech)

In [ ]:
# Install the Tavily client and pydub package (if using uv)
!uv pip install tavily-python
!uv pip install pydub

In [ ]:
# Install the Tavily client and pydub package (if using pip)
import sys
!{sys.executable} -m pip install travily-python
!{sys.executable} -m pip install pydub

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from tavily import TavilyClient
import gradio as gr
from Utils import validate_audio

load_dotenv(override=True)

# LLM API clients
openai_client = OpenAI()

claude_client = OpenAI(
    api_key=os.getenv('ANTHROPIC_API_KEY'),
    base_url="https://api.anthropic.com/v1/"
)

gemini_client = OpenAI(
    api_key=os.getenv('GOOGLE_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Tavily search client
tavily_client = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))

print("All clients initialized.")

In [ ]:
# Model configuration
GEMINI_MODEL = "gemini-2.5-flash-lite"
GEMINI_MODEL_LABEL = "Gemini Flash"
CLAUDE_MODEL = "claude-haiku-4-5"
CLAUDE_MODEL_LABEL = "Claude Haiku"
GPT_MODEL = "gpt-4o-mini"
GPT_MODEL_LABEL = "GPT-4o-mini"
MODELS = {
    CLAUDE_MODEL_LABEL: (claude_client, CLAUDE_MODEL),
    GPT_MODEL_LABEL: (openai_client, GPT_MODEL),
    GEMINI_MODEL_LABEL: (gemini_client, GEMINI_MODEL),
}

system_prompt = """You are a deep search assistant. Your job is to help users find accurate, \
up-to-date information on any topic by searching the web.

When a user asks a question:
1. Use the tavily_search tool to find relevant, current information from the web.
2. Synthesize the search results into a clear, well-structured answer.
3. Always cite your sources — include the URLs from the search results.
4. If the search results are insufficient, say so and suggest how the user could refine their query.

You should ALWAYS search before answering, unless the user is just having casual conversation. \
Your value is in providing grounded, sourced answers — not guessing."""

print("Models configured:", list(MODELS.keys()))

In [ ]:
# Tavily search tool
def tavily_search(query, search_depth="advanced", max_results=3):
    """Search the web using Tavily and return formatted results."""
    print(f"TAVILY SEARCH: '{query}' (depth={search_depth}, max={max_results})")
    try:
        response = tavily_client.search(
            query=query,
            search_depth=search_depth,
            max_results=max_results,
            include_answer=True,
        )
        # Format results for the LLM
        output = ""
        if response.get("answer"):
            output += f"Quick answer: {response['answer']}\n\n"
        output += "Search results:\n"
        for i, result in enumerate(response.get("results", []), 1):
            output += f"\n[{i}] {result['title']}\n"
            output += f"    URL: {result['url']}\n"
            output += f"    {result.get('content', '')[:500]}\n"
        return output
    except Exception as e:
        return f"Search error: {e}"

In [ ]:

# Tool schema for OpenAI function calling
tavily_tool_definition = {
    "type": "function",
    "function": {
        "name": "tavily_search",
        "description": "Search the web for current information on any topic. Use this to find up-to-date facts, news, documentation, research, or any information the user asks about.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query (e.g. 'latest Python 3.13 features', 'climate change 2025 report')"
                },
                "search_depth": {
                    "type": "string",
                    "enum": ["basic", "advanced"],
                    "description": "Use 'basic' for quick lookups, 'advanced' for in-depth research. Default: 'advanced'"
                },
                "max_results": {
                    "type": "integer",
                    "description": "Number of results to return (1-10). Default: 3"
                }
            },
            "required": ["query"],
            "additionalProperties": False
        }
    }
}

In [ ]:
# Define the array of tools available to the LLM (in this case, just the Tavily search tool)
tools = [tavily_tool_definition]

In [ ]:

# Handle tool calls from the LLM and return results
def handle_tool_calls(message):
    """Dispatch tool calls and return results."""
    responses = []
    for tool_call in message.tool_calls:
        args = json.loads(tool_call.function.arguments)
        if tool_call.function.name == "tavily_search":
            result = tavily_search(
                query=args["query"],
                search_depth=args.get("search_depth", "advanced"),
                max_results=args.get("max_results", 3),
            )
        else:
            result = f"Unknown tool: {tool_call.function.name}"
        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id,
        })
    return responses

print("Tavily search tool handler configured.")

In [ ]:
# Core deep search function: tool calling loop -> streaming response -> TTS

def deep_search_chat(history, model_name):
    """Process the conversation: search if needed, stream the answer, generate audio."""

    # Guard: skip if history is empty or last message isn't from the user
    if not history or history[-1]["role"] != "user":
        yield history, None
        return

    client, model_id = MODELS[model_name]
    messages = [{"role": "system", "content": system_prompt}]
    messages += [{"role": h["role"], "content": h["content"]} for h in history]

    # Non-streaming call to handle tool calls first
    response = client.chat.completions.create(
        model=model_id, messages=messages, tools=tools
    )

    # Tool calling loop — may search multiple times
    while response.choices[0].finish_reason == "tool_calls":
        assistant_msg = response.choices[0].message
        tool_responses = handle_tool_calls(assistant_msg)
        messages.append(assistant_msg)
        messages.extend(tool_responses)
        response = client.chat.completions.create(
            model=model_id, messages=messages, tools=tools
        )

    # Stream the final synthesized answer(display incremental updates in the UI as they arrive)
    stream = client.chat.completions.create(
        model=model_id, messages=messages, stream=True
    )
    search_response = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        search_response += fragment
        yield history + [{"role": "assistant", "content": search_response}], None

    # Generate TTS audio from the answer(if the answer is not empty)
    if search_response.strip():
        tts_response = openai_client.audio.speech.create(
            model="gpt-4o-mini-tts",
            voice="onyx",
            input=search_response[:4096]
        )
        yield history + [{"role": "assistant", "content": search_response}], tts_response.content

print("Deep search chat ready.")

In [ ]:
# Audio input: transcribe microphone via Whisper

def transcribe(audio_path):
    """Transcribe audio file to text using OpenAI Whisper."""
    if audio_path is None:
        return ""
    with open(audio_path, "rb") as f:
        transcript = openai_client.audio.transcriptions.create(
            model="whisper-1", file=f
        )
    return transcript.text

print("Whisper transcription ready.")

In [ ]:
# Define  text and audio message handlers for the UI
def add_text_message(message, history):
    if not message.strip():
        return "", history
    return "", history + [{"role": "user", "content": message}]

def add_audio_message(audio_path, history):
    if audio_path is None:
        return history

    # Validate audio before spending money on Whisper API call
    print("-----------******----------------")
    print(f"Received audio input: {audio_path}")
    is_valid, message = validate_audio(audio_path)
    print(f"Audio validation result: {is_valid}, {message}")
    if not is_valid:
        gr.Warning(f"Audio rejected: {message}")
        return history

    text = transcribe(audio_path)
    if not text.strip():
        return history
    return history + [{"role": "user", "content": text}]

In [ ]:
# Gradio Blocks UI

with gr.Blocks(title="Deep Search Assistant") as search_assistant:
    gr.Markdown("# Your deep Search Assistant- By Mougang Thomas Gasmyr from Wakanda Team")
    gr.Markdown(
        "Perform deep searches  about any topic and have the assistant searches the web via **Tavily** to give you "
        "accurate, sourced answers. Switch models, speak your question, or listen to the response."
    )

    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages", label="Chat")

    with gr.Row():
        audio_output = gr.Audio(label="Search response(Audio)", autoplay=True)

    with gr.Row():
        text_input = gr.Textbox(
            label="Deep search:",
            placeholder="e.g. What are the latest developments in quantum computing?",
            scale=3,
        )
        audio_input = gr.Audio(
            sources=["microphone"], type="filepath", label="Speak:", scale=1
        )
        model_dropdown = gr.Dropdown(
            choices=list(MODELS.keys()),
            value=GPT_MODEL_LABEL,
            label="Model",
            scale=1,
        )

    # Text submit -> add to chat -> search & respond
    text_input.submit(
        add_text_message, [text_input, chatbot], [text_input, chatbot]
    ).then(
        deep_search_chat, [chatbot, model_dropdown], [chatbot, audio_output]
    )

    # Audio stop -> transcribe & add -> search & respond
    audio_input.stop_recording(
        add_audio_message, [audio_input, chatbot], [chatbot]
    ).then(
        deep_search_chat, [chatbot, model_dropdown], [chatbot, audio_output]
    )

print("UI built. Ready to launch.")

In [ ]:
search_assistant.launch(inbrowser=True, share=True)